<a href="https://colab.research.google.com/github/mrjooj/Estudos/blob/main/lista_de_compras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import re

# Lista de compras: cada elemento é [nome, categoria, valor]
lista_compras = []

# Categorias permitidas (vetor)
categorias = ["eletrodomésticos", "itens de limpeza", "comida", "aparelhos eletrônicos"]

# Emojis para cada categoria (para exibição)
emojis_cat = {
    "eletrodomésticos": "🧊",
    "itens de limpeza": "🧹",
    "comida": "🍎",
    "aparelhos eletrônicos": "📱"
}

# Função para validar e converter valor para float
def parse_valor(texto):
    texto = texto.strip().replace(',', '.').replace('R$', '').strip()
    try:
        valor = float(texto)
        return round(valor, 2)
    except ValueError:
        return None

# Função para formatar valor em reais
def formatar_valor(valor):
    return f"R$ {valor:.2f}".replace('.', ',')

# Função para atualizar a exibição da lista
def atualizar_lista():
    with output_area:
        clear_output(wait=True)
        if not lista_compras:
            display(widgets.HTML("<p style='color: gray; font-size: 18px;'>🛒 Sua lista de compras está vazia.</p>"))
        else:
            # Agrupar itens por categoria e calcular totais
            itens_por_categoria = {cat: [] for cat in categorias}
            total_geral = 0.0
            for item, cat, valor in lista_compras:
                itens_por_categoria[cat].append((item, valor))
                total_geral += valor

            # Construir HTML estilizado (como um carrinho de compras)
            html = "<div style='font-family: Arial, sans-serif; max-width: 650px; margin: 10px auto; border: 1px solid #ddd; border-radius: 8px; padding: 15px; background-color: #f9f9f9; box-shadow: 0 2px 5px rgba(0,0,0,0.1);'>"
            html += "<h2 style='text-align: center; color: #333; border-bottom: 2px solid #ff6600; padding-bottom: 10px;'>🛍️ SEU CARRINHO</h2>"

            for cat in categorias:
                itens = itens_por_categoria[cat]
                if itens:
                    emoji = emojis_cat.get(cat, "📦")
                    subtotal = sum(valor for _, valor in itens)
                    html += f"<div style='margin-top: 20px;'><b style='font-size: 1.1em;'>{emoji} {cat.upper()}</b> <span style='color: #666;'>({len(itens)} itens) - Subtotal: <span style='color: #28a745;'>{formatar_valor(subtotal)}</span></span>"
                    html += "<ul style='list-style-type: none; padding-left: 15px; margin-top: 5px;'>"
                    for i, (item, valor) in enumerate(itens, 1):
                        html += f"<li style='margin: 8px 0; border-bottom: 1px dashed #ccc; padding-bottom: 3px;'>{i}. {item} - <span style='color: #28a745; font-weight: bold;'>{formatar_valor(valor)}</span></li>"
                    html += "</ul></div>"

            # Total geral
            html += f"<div style='margin-top: 25px; text-align: right; font-size: 1.3em; border-top: 2px solid #333; padding-top: 10px;'><b>TOTAL: <span style='color: #28a745;'>{formatar_valor(total_geral)}</span></b></div>"
            html += "</div>"

            display(widgets.HTML(html))

        # Atualizar dropdown de remoção com os itens atuais (incluindo valor)
        dropdown_remover.options = [(f"{item[0]} - {item[1]} - {formatar_valor(item[2])}", idx) for idx, item in enumerate(lista_compras)]

# Callback do botão adicionar
def adicionar_item(b):
    nome = input_nome.value.strip()
    if not nome:
        with output_controle:
            clear_output()
            display(widgets.HTML("<p style='color: red;'>❌ Nome inválido. Digite um nome.</p>"))
        return

    valor_texto = input_valor.value.strip()
    valor = parse_valor(valor_texto)
    if valor is None or valor < 0:
        with output_controle:
            clear_output()
            display(widgets.HTML("<p style='color: red;'>❌ Valor inválido. Digite um número positivo (ex: 19.99 ou 19,99).</p>"))
        return

    categoria = dropdown_categoria.value
    lista_compras.append([nome, categoria, valor])

    # Limpar campos e feedback
    input_nome.value = ""
    input_valor.value = ""
    with output_controle:
        clear_output()
        display(widgets.HTML(f"<p style='color: green;'>✅ Item '{nome}' adicionado por {formatar_valor(valor)} à categoria '{categoria}'.</p>"))

    atualizar_lista()

# Callback do botão remover
def remover_item(b):
    if not lista_compras:
        with output_controle:
            clear_output()
            display(widgets.HTML("<p style='color: red;'>❌ Lista vazia. Nada a remover.</p>"))
        return

    indice = dropdown_remover.value
    if indice is None:
        with output_controle:
            clear_output()
            display(widgets.HTML("<p style='color: red;'>❌ Selecione um item para remover.</p>"))
        return

    removido = lista_compras.pop(indice)
    with output_controle:
        clear_output()
        display(widgets.HTML(f"<p style='color: green;'>✅ Item '{removido[0]}' removido.</p>"))

    atualizar_lista()

# Callback do botão atualizar (força recarregar a lista)
def atualizar_manual(b):
    atualizar_lista()
    with output_controle:
        clear_output()
        display(widgets.HTML("<p style='color: blue;'>🔄 Lista atualizada.</p>"))

# Criação dos widgets
titulo = widgets.HTML("<h1 style='text-align: center; color: #ff6600; font-family: Arial;'>🛍️  MERCADO PYTHON  🛍️</h1><p style='text-align: center; color: gray;'>Seu carrinho de compras inteligente</p>")

# Seção de adicionar
input_nome = widgets.Text(
    placeholder="Ex: Geladeira, Arroz, Sabão...",
    description="Item:",
    layout=widgets.Layout(width='300px')
)
input_valor = widgets.Text(
    placeholder="Ex: 1299.90 ou 19,99",
    description="Valor (R$):",
    layout=widgets.Layout(width='200px')
)
dropdown_categoria = widgets.Dropdown(
    options=categorias,
    value=categorias[0],
    description="Categoria:",
    layout=widgets.Layout(width='300px')
)
botao_adicionar = widgets.Button(
    description="Adicionar ao carrinho",
    button_style='success',
    icon='plus',
    layout=widgets.Layout(width='200px')
)
botao_adicionar.on_click(adicionar_item)

# Seção de remover
dropdown_remover = widgets.Dropdown(
    options=[],  # Preenchido dinamicamente
    description="Remover item:",
    layout=widgets.Layout(width='450px')
)
botao_remover = widgets.Button(
    description="Remover do carrinho",
    button_style='danger',
    icon='trash',
    layout=widgets.Layout(width='200px')
)
botao_remover.on_click(remover_item)

# Botão para atualizar lista manualmente
botao_atualizar = widgets.Button(
    description="Atualizar carrinho",
    button_style='info',
    icon='refresh',
    layout=widgets.Layout(width='200px')
)
botao_atualizar.on_click(atualizar_manual)

# Áreas de saída
output_controle = widgets.Output(layout=widgets.Layout(min_height='50px'))
output_area = widgets.Output()

# Layout principal usando VBox e HBox
controle_adicionar = widgets.HBox([input_nome, input_valor, dropdown_categoria, botao_adicionar])
controle_remover = widgets.HBox([dropdown_remover, botao_remover])
controle_atualizar = widgets.HBox([botao_atualizar])

interface = widgets.VBox([
    titulo,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>Adicionar item</h3>"),
    controle_adicionar,
    widgets.HTML("<h3>Remover item</h3>"),
    controle_remover,
    controle_atualizar,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>Mensagens</h3>"),
    output_controle,
    widgets.HTML("<h3>Carrinho de Compras</h3>"),
    output_area
])

# Exibir a interface
display(interface)

# Atualizar lista inicial (vazia)
atualizar_lista()